In [ ]:
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

# get total using memory

In [ ]:
def get_used_memory_MB():
    return np.round(psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2,2)

# get datafame memory

In [ ]:
def get_used_memory_df(df):
    return np.round(df.memory_usage().sum() / 1024 ** 2,2)

# reduce memory fuction

In [ ]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

# load data

In [ ]:
print('Using total Memory=', get_used_memory_MB(),"MB")

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-oct-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-oct-2021/test.csv")

In [ ]:
print('Using total Memory=', get_used_memory_MB(),"MB")

# reduce data

In [ ]:
print("train memroy = ",get_used_memory_df(train),"MB")
train_reduce = reduce_memory_usage(train)
print("test memory = ",get_used_memory_df(train_reduce),"MB")

In [ ]:
print("train memroy = ",get_used_memory_df(test),"MB")
test_reduce = reduce_memory_usage(test)
print("test memory = ",get_used_memory_df(test_reduce),"MB")

In [ ]:
print('Using total Memory=', get_used_memory_MB(),"MB")

# garbage collect

In [ ]:
del train
del test
gc.collect()

In [ ]:
print("train memroy = ",get_used_memory_df(train_reduce),"MB")
print("test memory = ",get_used_memory_df(test_reduce),"MB")

In [ ]:
print('Using total Memory=', get_used_memory_MB(),"MB")